<a href="https://colab.research.google.com/github/SpectraGbes/Spectragbes/blob/main/End%E2%80%91to%E2%80%91End_Option_Pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from math import exp, sqrt, log
from numpy.linalg import solve
from numpy.random import default_rng

In [2]:
# Display utility (if available)
try:
    from caas_jupyter_tools import display_dataframe_to_user
except Exception:
    display_dataframe_to_user = None

# File names provided by the user
files = {
    'SEA': 'SEA.csv',
    'AMKBY': 'AMKBY.csv'
}

# --- Helper functions for volatility estimators ---

def prep_df(df):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date').reset_index(drop=True)
    # Prefer Adj Close when available, else Close
    if 'Adj Close' in df.columns:
        df['AdjClose'] = pd.to_numeric(df['Adj Close'], errors='coerce')
    else:
        df['AdjClose'] = pd.to_numeric(df['Close'], errors='coerce')
    for col in ['Open','High','Low','Close']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    # Daily log-returns (close-to-close, using adjusted close)
    df['logret'] = np.log(df['AdjClose']).diff()
    # Overnight (open/prev close) and intraday (close/open)
    df['overnight'] = np.log(df['Open'] / df['AdjClose'].shift(1))
    df['oc'] = np.log(df['Close'] / df['Open'])
    # Components for range estimators
    df['hl'] = np.log(df['High'] / df['Low'])
    # Rogers-Satchell per-day variance component
    df['rs'] = np.log(df['High']/df['Open']) * np.log(df['High']/df['Close']) + \
               np.log(df['Low']/df['Open'])  * np.log(df['Low']/df['Close'])
    return df

TRADING_DAYS = 252

# Close-to-Close (standard realized vol)
def realized_vol_close_to_close(logrets):
    r = logrets.dropna()
    if len(r) < 2:
        return np.nan
    return np.sqrt(TRADING_DAYS) * r.std(ddof=1)

# Parkinson (uses high/low range)
def parkinson_vol(hl):
    x = hl.dropna()
    if len(x) == 0:
        return np.nan
    var = (x**2).mean() / (4 * np.log(2))
    return np.sqrt(TRADING_DAYS * var)

# Garman-Klass (OHLC)
def garman_klass_vol(df):
    x = df.dropna(subset=['hl','oc'])
    if len(x) == 0:
        return np.nan
    var = 0.5*(x['hl']**2).mean() - (2*np.log(2)-1)*(x['oc']**2).mean()
    return np.sqrt(TRADING_DAYS * var)

# Rogers-Satchell (OHLC, drift-independent)
def rogers_satchell_vol(rs):
    x = rs.dropna()
    if len(x) == 0:
        return np.nan
    var = x.mean()
    return np.sqrt(TRADING_DAYS * var)

# Yang-Zhang (combines overnight, oc, and RS)
def yang_zhang_vol(df):
    x = df.dropna(subset=['overnight','oc','rs'])
    n = len(x)
    if n < 2:
        return np.nan
    k = 0.34 / (1.34 + (n + 1) / (n - 1))
    var_overnight = x['overnight'].var(ddof=1)
    var_oc = x['oc'].var(ddof=1)
    rs_mean = x['rs'].mean()
    var = var_overnight + k*var_oc + (1 - k)*rs_mean
    return np.sqrt(TRADING_DAYS * var)

# Rolling Yang-Zhang helper
def rolling_yz(df, window=21):
    out = []
    for i in range(len(df)):
        lo = max(0, i - window + 1)
        sub = df.iloc[lo:i+1]
        vol = yang_zhang_vol(sub)
        out.append(vol)
    return pd.Series(out, index=df.index)

# Container for summaries and rolling series
summaries = []
rolling_plots = {}

for symbol, fname in files.items():
    path = Path(fname)
    if not path.exists():
        raise FileNotFoundError(f"File not found: {fname}")
    raw = pd.read_csv(path)
    df = prep_df(raw)
    df['Symbol'] = symbol

    # Full-sample estimators
    vol_cc = realized_vol_close_to_close(df['logret'])
    vol_pk = parkinson_vol(df['hl'])
    vol_gk = garman_klass_vol(df)
    vol_rs = rogers_satchell_vol(df['rs'])
    vol_yz = yang_zhang_vol(df)

    # Rolling (last 21, 63, 126 trading days) using YZ and CC where feasible
    def windowed(vol_func, series, w):
        s = series.dropna()
        if len(s) >= w:
            return np.sqrt(TRADING_DAYS) * s.tail(w).std(ddof=1)
        return np.nan

    last21_cc = windowed(lambda x: x, df['logret'], 21)
    last63_cc = windowed(lambda x: x, df['logret'], 63)
    last126_cc = windowed(lambda x: x, df['logret'], 126)

    # Rolling YZ time series for plotting (21-day)
    df['YZ_21d'] = rolling_yz(df, window=21)

    # Collect summary row
    summaries.append({
        'Symbol': symbol,
        'Start': df['Date'].min().date(),
        'End': df['Date'].max().date(),
        'Observations': int(df['Date'].nunique()),
        'Mean daily log-return': df['logret'].mean(skipna=True),
        'Daily std (log-return)': df['logret'].std(skipna=True, ddof=1),
        'Annualized Vol (Close-Close)': vol_cc,
        'Annualized Vol (Parkinson)': vol_pk,
        'Annualized Vol (Garman-Klass)': vol_gk,
        'Annualized Vol (Rogers-Satchell)': vol_rs,
        'Annualized Vol (Yang-Zhang)': vol_yz,
        '21d Vol (Close-Close)': last21_cc,
        '63d Vol (Close-Close)': last63_cc,
        '126d Vol (Close-Close)': last126_cc,
    })

    # Save per-symbol enriched dataset
    out_csv = f"{symbol}_enriched_with_returns.csv"
    df.to_csv(out_csv, index=False)

    # Prepare plot of rolling 21d YZ
    plt.figure(figsize=(8, 3))
    plt.plot(df['Date'], df['YZ_21d'], label='21d YZ vol (annualized)')
    plt.title(f"{symbol}: Rolling 21-day Yang–Zhang Volatility (Annualized)")
    plt.xlabel('Date')
    plt.ylabel('Volatility')
    plt.grid(True, alpha=0.3)
    plt.legend()
    fig_path = f"{symbol}_rolling_YZ_21d.png"
    plt.tight_layout()
    plt.savefig(fig_path, dpi=150)
    plt.close()

# Summary dataframe
summary_df = pd.DataFrame(summaries)
summary_df = summary_df[['Symbol','Start','End','Observations',
                         'Mean daily log-return','Daily std (log-return)',
                         'Annualized Vol (Close-Close)','Annualized Vol (Parkinson)',
                         'Annualized Vol (Garman-Klass)','Annualized Vol (Rogers-Satchell)',
                         'Annualized Vol (Yang-Zhang)',
                         '21d Vol (Close-Close)','63d Vol (Close-Close)','126d Vol (Close-Close)']]

# Save summary
summary_csv = 'volatility_summary.csv'
summary_df.to_csv(summary_csv, index=False)

# Display to user if possible
if display_dataframe_to_user:
    display_dataframe_to_user("Volatility Summary (SEA & AMKBY)", summary_df)

summary_df.head().to_dict(orient='records')

[{'Symbol': 'SEA',
  'Start': datetime.date(2026, 1, 5),
  'End': datetime.date(2026, 2, 6),
  'Observations': 24,
  'Mean daily log-return': 0.00482311133509919,
  'Daily std (log-return)': 0.010030861141613492,
  'Annualized Vol (Close-Close)': 0.15923498409918452,
  'Annualized Vol (Parkinson)': 0.11300801506235394,
  'Annualized Vol (Garman-Klass)': 0.1176951431779175,
  'Annualized Vol (Rogers-Satchell)': 0.13053901228022224,
  'Annualized Vol (Yang-Zhang)': 0.19793607742877695,
  '21d Vol (Close-Close)': 0.16696269992852303,
  '63d Vol (Close-Close)': nan,
  '126d Vol (Close-Close)': nan},
 {'Symbol': 'AMKBY',
  'Start': datetime.date(2025, 8, 11),
  'End': datetime.date(2026, 2, 6),
  'Observations': 121,
  'Mean daily log-return': 0.0009557174928820837,
  'Daily std (log-return)': 0.023291408794295105,
  'Annualized Vol (Close-Close)': 0.36973965212428567,
  'Annualized Vol (Parkinson)': 0.1391955610999998,
  'Annualized Vol (Garman-Klass)': 0.13757302386520515,
  'Annualized V

In [3]:
# Load enriched files to get last non-NaN YZ 21d values
symbols = ['SEA','AMKBY']
last_vals = {}
for s in symbols:
    df = pd.read_csv(f'{s}_enriched_with_returns.csv')
    val = pd.to_numeric(df['YZ_21d'], errors='coerce').dropna()
    last_vals[s] = float(val.iloc[-1]) if len(val)>0 else np.nan
last_vals


{'SEA': 0.2029992290715125, 'AMKBY': 0.4680014954651696}

In [4]:
TRADING_DAYS = 252

# Inputs from prior step / user
r = 0.05  # 5% risk-free
q = 0.0   # 0% dividend yield

# Load enriched data to get
# - latest price (AdjClose)
# - latest 21d YZ vol as near-term sigma
symbols = ['SEA','AMKBY']
market = {}
for s in symbols:
    df = pd.read_csv(f'{s}_enriched_with_returns.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    S0 = float(pd.to_numeric(df['AdjClose'], errors='coerce').dropna().iloc[-1])
    # latest non-NaN 21d YZ vol
    sigma = float(pd.to_numeric(df['YZ_21d'], errors='coerce').dropna().iloc[-1])
    market[s] = {'S0': S0, 'sigma': sigma}

# Helper: Black-Scholes formula for European call/put and Greeks (for benchmarking)
from scipy.stats import norm

def bs_price_greeks(S, K, T, r, q, sigma, option='call'):
    if T <= 0 or sigma <= 0:
        # intrinsic at expiry
        if option=='call':
            price = max(0.0, S - K)
            delta = 1.0 if S>K else (0.0 if S<K else 0.5)
        else:
            price = max(0.0, K - S)
            delta = -1.0 if S<K else (0.0 if S>K else -0.5)
        return {'price': price, 'delta': delta, 'gamma': 0.0, 'theta': 0.0, 'vega': 0.0, 'rho': 0.0}
    d1 = (np.log(S/K) + (r - q + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    if option=='call':
        price = S*np.exp(-q*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
        delta = np.exp(-q*T)*norm.cdf(d1)
        rho = K*T*np.exp(-r*T)*norm.cdf(d2)
    else:
        price = K*np.exp(-r*T)*norm.cdf(-d2) - S*np.exp(-q*T)*norm.cdf(-d1)
        delta = -np.exp(-q*T)*norm.cdf(-d1)
        rho = -K*T*np.exp(-r*T)*norm.cdf(-d2)
    gamma = np.exp(-q*T) * norm.pdf(d1) / (S*sigma*np.sqrt(T))
    vega = S*np.exp(-q*T)*norm.pdf(d1)*np.sqrt(T)
    theta = -(S*np.exp(-q*T)*norm.pdf(d1)*sigma)/(2*np.sqrt(T)) - (r*K*np.exp(-r*T)* (norm.cdf(d2) if option=='call' else -norm.cdf(-d2))) + q*S*np.exp(-q*T)*(norm.cdf(d1) if option=='call' else -norm.cdf(-d1))
    return {'price': float(price), 'delta': float(delta), 'gamma': float(gamma), 'theta': float(theta), 'vega': float(vega), 'rho': float(rho)}

# CRR Binomial (supports American early exercise)

def crr_tree_price(S0, K, T, r, q, sigma, N=200, option='call', american=False):
    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    a = np.exp((r-q)*dt)
    p = (a - d)/(u - d)
    disc = np.exp(-r*dt)

    # stock prices at maturity
    ST = np.array([S0 * (u**j) * (d**(N-j)) for j in range(N+1)])
    if option=='call':
        V = np.maximum(0.0, ST - K)
    else:
        V = np.maximum(0.0, K - ST)

    # backward induction
    for i in range(N-1, -1, -1):
        ST = ST[:i+1] / d  # move one step back
        V = disc * (p*V[1:] + (1-p)*V[:-1])
        if american:
            if option=='call' and q==0:
                # For non-dividend stocks, early exercise of calls is suboptimal; but we keep generic formula
                exercise = np.maximum(0.0, ST - K)
            elif option=='call':
                exercise = np.maximum(0.0, ST - K)
            else:
                exercise = np.maximum(0.0, K - ST)
            V = np.maximum(V, exercise)
    return float(V[0])

# Tree Greeks via bump-and-reprice (robust across Euro/American)

def tree_greeks(S0, K, T, r, q, sigma, N, option='call', american=False, dS=0.01, dT=1/252, dV=0.0001, dr=0.0001):
    # Base
    P0 = crr_tree_price(S0, K, T, r, q, sigma, N, option, american)
    # Delta & Gamma
    P_up = crr_tree_price(S0+dS, K, T, r, q, sigma, N, option, american)
    P_dn = crr_tree_price(S0-dS, K, T, r, q, sigma, N, option, american)
    delta = (P_up - P_dn)/(2*dS)
    gamma = (P_up - 2*P0 + P_dn)/(dS**2)
    # Theta (1-day forward difference)
    T_dn = max(1e-6, T - dT)
    P_Tdn = crr_tree_price(S0, K, T_dn, r, q, sigma, N, option, american)
    theta = (P_Tdn - P0) / (-dT)  # per year
    # Vega
    P_sv_up = crr_tree_price(S0, K, T, r, q, sigma+dV, N, option, american)
    P_sv_dn = crr_tree_price(S0, K, T, r, q, sigma-dV, N, option, american)
    vega = (P_sv_up - P_sv_dn) / (2*dV)
    # Rho
    P_r_up = crr_tree_price(S0, K, T, r+dr, q, sigma, N, option, american)
    P_r_dn = crr_tree_price(S0, K, T, r-dr, q, sigma, N, option, american)
    rho = (P_r_up - P_r_dn) / (2*dr)
    return {'price': P0, 'delta': delta, 'gamma': gamma, 'theta': theta, 'vega': vega, 'rho': rho}

# Simple Kamrad–Ritchken trinomial tree for European and American

def kr_trinomial_price(S0, K, T, r, q, sigma, N=200, option='call', american=False):
    dt = T/N
    nu = r - q - 0.5*sigma**2
    dx = sigma*np.sqrt(3*dt)
    u = np.exp(dx)
    d = np.exp(-dx)
    pu = 1/6 + (nu*np.sqrt(dt))/(2*dx) + ( (sigma**2)*dt )/ (2*dx**2)
    pm = 2/3 - ( (sigma**2)*dt )/ (dx**2)
    pd = 1 - pu - pm
    disc = np.exp(-r*dt)

    # Grid size grows with N; we use arrays indexed from -i..i
    # Initialize terminal payoffs
    m = 2*N+1
    prices = np.zeros(m)
    # indices shift so that center corresponds to j=N (0 moves)
    for k in range(-N, N+1):
        S = S0 * np.exp(k*dx)
        payoff = max(0.0, S-K) if option=='call' else max(0.0, K-S)
        prices[k+N] = payoff

    # Backward induction
    for i in range(N-1, -1, -1):
        new_prices = np.zeros(2*i+1)
        for k in range(-i, i+1):
            idx = k + i
            cont = disc*(pu*prices[idx+2] + pm*prices[idx+1] + pd*prices[idx])
            if american:
                S = S0 * np.exp(k*dx)
                exercise = max(0.0, S-K) if option=='call' else max(0.0, K-S)
                new_prices[idx] = max(cont, exercise)
            else:
                new_prices[idx] = cont
        prices = new_prices
    return float(prices[0])

# Monte Carlo (GBM) for European options and pathwise delta estimate; LSMC could be added later for American.

def mc_gbm_european(S0, K, T, r, q, sigma, option='call', n_paths=100000, n_steps=252, antithetic=True, seed=42):
    rng = np.random.default_rng(seed)
    dt = T/n_steps
    drift = (r - q - 0.5*sigma**2)*dt
    vol = sigma*np.sqrt(dt)

    n_sim = n_paths//2 if antithetic else n_paths
    Z = rng.standard_normal((n_sim, n_steps))
    if antithetic:
        Z = np.vstack([Z, -Z])

    S = np.full((n_paths,), S0, dtype=float)
    for t in range(n_steps):
        S *= np.exp(drift + vol*Z[:,t])

    if option=='call':
        payoff = np.maximum(S - K, 0.0)
    else:
        payoff = np.maximum(K - S, 0.0)
    disc = np.exp(-r*T)
    price = disc * payoff.mean()

    # Standard error
    se = disc * payoff.std(ddof=1) / np.sqrt(n_paths)

    # Pathwise delta for European call/put under GBM
    # Using likelihood ratio for put/call could be added; for brevity compute via bump here
    bump = 0.01
    S_bumped = np.full((n_paths,), S0 + bump, dtype=float)
    S_b = S_bumped.copy()
    for t in range(n_steps):
        S_b *= np.exp(drift + vol*Z[:,t])
    if option=='call':
        payoff_b = np.maximum(S_b - K, 0.0)
    else:
        payoff_b = np.maximum(K - S_b, 0.0)
    price_b = disc * payoff_b.mean()
    delta_mc = (price_b - price)/bump

    return {'price': float(price), 'stderr': float(se), 'delta_mc_bump': float(delta_mc)}

# Build scenarios (ATM strikes at 30d, 60d, 90d)
scenarios = []
terms_days = [30, 60, 90]
for s, info in market.items():
    S0 = info['S0']
    sigma = info['sigma']
    for days in terms_days:
        T = days/365.0
        K = round(S0, 2)  # ATM at last price, cents
        # Prices and Greeks from CRR (American & European)
        N = max(200, int(100*sqrt(days/30)))
        for opt in ['call','put']:
            euro_crr = tree_greeks(S0, K, T, r, q, sigma, N, option=opt, american=False)
            amer_crr = tree_greeks(S0, K, T, r, q, sigma, N, option=opt, american=True)
            # Trinomial prices (no Greeks here to keep runtime reasonable)
            euro_tri = kr_trinomial_price(S0, K, T, r, q, sigma, N, option=opt, american=False)
            amer_tri = kr_trinomial_price(S0, K, T, r, q, sigma, N, option=opt, american=True)
            # Monte Carlo for European (benchmark price + stderr)
            mc = mc_gbm_european(S0, K, T, r, q, sigma, option=opt, n_paths=50000, n_steps=max(50, days), antithetic=True)
            # Black-Scholes for European (sanity check)
            bs = bs_price_greeks(S0, K, T, r, q, sigma, option=opt)

            scenarios.append({
                'Symbol': s,
                'S0': S0,
                'Sigma (annual)': sigma,
                'Days': days,
                'T_years': T,
                'Strike': K,
                'Option': opt,
                'CRR_Euro_Price': euro_crr['price'],
                'CRR_Euro_Delta': euro_crr['delta'],
                'CRR_Euro_Gamma': euro_crr['gamma'],
                'CRR_Euro_Theta/yr': euro_crr['theta'],
                'CRR_Euro_Vega': euro_crr['vega'],
                'CRR_Euro_Rho': euro_crr['rho'],
                'CRR_Amer_Price': amer_crr['price'],
                'CRR_Amer_Delta': amer_crr['delta'],
                'CRR_Amer_Gamma': amer_crr['gamma'],
                'CRR_Amer_Theta/yr': amer_crr['theta'],
                'CRR_Amer_Vega': amer_crr['vega'],
                'CRR_Amer_Rho': amer_crr['rho'],
                'TRI_Euro_Price': euro_tri,
                'TRI_Amer_Price': amer_tri,
                'MC_Euro_Price': mc['price'],
                'MC_StdErr': mc['stderr'],
                'MC_Delta_bump': mc['delta_mc_bump'],
                'BS_Euro_Price': bs['price'],
                'BS_Euro_Delta': bs['delta'],
                'BS_Euro_Gamma': bs['gamma'],
                'BS_Euro_Theta/yr': bs['theta'],
                'BS_Euro_Vega': bs['vega'],
                'BS_Euro_Rho': bs['rho']
            })

# Save pricing table
pricing_df = pd.DataFrame(scenarios)
pricing_df.to_csv('option_pricing_outputs.csv', index=False)

# Simple delta-hedging replication demo on simulated path (European call, ATM, 60d)
# We'll use SEA parameters
S0 = market['SEA']['S0']
sigma = market['SEA']['sigma']
K = round(S0, 2)
T = 60/365
steps = 60
N = 300

# Build CRR deltas along the tree by bumping prices at each node (for path-based hedging, we simulate GBM and recompute delta each day via BS for speed)
from numpy.random import default_rng
rng = default_rng(123)

dt = T/steps
S_path = [S0]
for t in range(steps):
    z = rng.standard_normal()
    S_next = S_path[-1]*np.exp((r - q - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*z)
    S_path.append(S_next)
S_path = np.array(S_path)

# Hedge rebalanced daily using Black-Scholes delta (fast; good approximation for Euro call)
opt = 'call'
shares = []
cash = []
portfolio = []

delta_prev = 0.0
cash_account = 0.0
for t, S in enumerate(S_path[:-1]):
    tau = T - t*dt
    greeks = bs_price_greeks(S, K, tau, r, q, sigma, option=opt)
    delta = greeks['delta']
    # rebalance shares
    d_shares = delta - delta_prev
    cash_account *= np.exp(r*dt)  # grow at risk-free
    cash_account -= d_shares * S  # buy shares if delta increases
    delta_prev = delta
    shares.append(delta)
    cash.append(cash_account)
    portfolio.append(delta*S + cash_account)

# Final step: settle option payoff and compare replication PnL
payoff = max(0.0, S_path[-1] - K)
# Final portfolio value after accruing interest one more step
cash_account *= np.exp(r*dt)
replication_value = delta_prev*S_path[-1] + cash_account
pnl = replication_value - payoff

# Plot hedging path
plt.figure(figsize=(9,4))
ax1 = plt.gca()
ax1.plot(S_path, label='Simulated Spot (SEA)')
ax1.set_xlabel('Day')
ax1.set_ylabel('Spot')
ax2 = ax1.twinx()
ax2.plot(shares, color='tab:orange', alpha=0.7, label='Delta (shares)')
ax2.set_ylabel('Delta')
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines+lines2, labels+labels2, loc='best')
plt.title('Delta-Hedging Simulation (European Call, 60d, SEA)')
plt.tight_layout()
plt.savefig('SEA_delta_hedging_sim.png', dpi=150)
plt.close()

summary = {
    'SEA': market['SEA'],
    'AMKBY': market['AMKBY'],
    'hedge_path_final_spot': float(S_path[-1]),
    'hedge_payoff': float(payoff),
    'hedge_replication_value': float(replication_value),
    'hedge_pnl': float(pnl)
}

pricing_df.head(6).to_dict(orient='records'), summary


([{'Symbol': 'SEA',
   'S0': 16.0,
   'Sigma (annual)': 0.2029992290715125,
   'Days': 30,
   'T_years': 0.0821917808219178,
   'Strike': 16.0,
   'Option': 'call',
   'CRR_Euro_Price': 0.4039372132331706,
   'CRR_Euro_Delta': 0.5396645208885026,
   'CRR_Euro_Gamma': 5.60690458332902,
   'CRR_Euro_Theta/yr': 2.6853092385041357,
   'CRR_Euro_Vega': 1.8186216007107237,
   'CRR_Euro_Rho': 0.6764954880558727,
   'CRR_Amer_Price': 0.4039372132331706,
   'CRR_Amer_Delta': 0.5396645208885026,
   'CRR_Amer_Gamma': 5.60690458332902,
   'CRR_Amer_Theta/yr': 2.6853092385041357,
   'CRR_Amer_Vega': 1.8186216007107237,
   'CRR_Amer_Rho': 0.6764954880558727,
   'TRI_Euro_Price': 2.1183635146960813,
   'TRI_Amer_Price': 2.1183635146960813,
   'MC_Euro_Price': 0.4014645903669221,
   'MC_StdErr': 0.0025904644771099447,
   'MC_Delta_bump': 0.5419307753617209,
   'BS_Euro_Price': 0.40440140615699605,
   'BS_Euro_Delta': 0.5397138594909758,
   'BS_Euro_Gamma': 0.4263070147902378,
   'BS_Euro_Theta/yr': -2

In [5]:
# --- Globals / inputs ---
r = 0.05
q = 0.0
rng = default_rng(20260208)

# Load latest S0 and sigma (from enriched files produced earlier)
market = {}
for s in ['SEA','AMKBY']:
    df = pd.read_csv(f'{s}_enriched_with_returns.csv')
    S0 = float(pd.to_numeric(df['AdjClose'], errors='coerce').dropna().iloc[-1])
    sigma = float(pd.to_numeric(df['YZ_21d'], errors='coerce').dropna().iloc[-1])
    market[s] = {'S0': S0, 'sigma': sigma}

# --- Black-Scholes for reference ---
from scipy.stats import norm

def bs_price(S, K, T, r, q, sigma, option='call'):
    if T<=0 or sigma<=0:
        return max(0.0, (S-K) if option=='call' else (K-S))
    d1 = (np.log(S/K) + (r - q + 0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    if option=='call':
        return float(S*np.exp(-q*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2))
    else:
        return float(K*np.exp(-r*T)*norm.cdf(-d2) - S*np.exp(-q*T)*norm.cdf(-d1))

# --- Improved Trinomial Tree (moment-matching on S) ---
# u = e^{σ√Δt}, m=1, d=1/u. Probabilities solve moment matching to E[S] and E[S^2].

def tri_probs_moment_match(dt, r, q, sigma):
    u = np.exp(sigma*np.sqrt(dt))
    d = 1.0/u
    # Target moments under risk-neutral GBM
    m1 = np.exp((r - q)*dt)              # E[S_{t+dt}/S_t]
    m2 = np.exp(2*(r - q)*dt + sigma**2 * dt)  # E[(S_{t+dt}/S_t)^2]
    # Solve linear system: [ [1,1,1],[u,1,d],[u^2,1,d^2] ] [pu, pm, pd]^T = [1, m1, m2]
    A = np.array([[1.0, 1.0, 1.0],
                  [u,   1.0, d  ],
                  [u*u, 1.0, d*d]])
    b = np.array([1.0, m1, m2])
    pu, pm, pd = solve(A, b)
    # Numerical guard: small negative/over-1 rounding -> clip and renormalize
    probs = np.array([pu, pm, pd])
    if np.any(probs < -1e-10) or np.any(probs > 1+1e-10):
        # Fallback to Kamrad–Ritchken with lambda=1 (dx = σ√(3 dt)) and clip
        dx = sigma*np.sqrt(3*dt)
        u = np.exp(dx); d = np.exp(-dx)
        nu = r - q - 0.5*sigma**2
        pu = 1/6 + (nu*np.sqrt(dt))/(2*dx) + (sigma**2*dt)/(2*dx*dx)
        pm = 2/3 - (sigma**2*dt)/(dx*dx)
        pd = 1 - pu - pm
        probs = np.clip(np.array([pu, pm, pd]), 0.0, 1.0)
        s = probs.sum()
        if s==0:
            probs = np.array([1/4, 1/2, 1/4])
        else:
            probs = probs / s
        u = np.exp(dx); d = np.exp(-dx)
        return u, 1.0, d, probs[0], probs[1], probs[2]
    return u, 1.0, d, float(pu), float(pm), float(pd)


def trinomial_price(S0, K, T, r, q, sigma, N=200, option='call', american=False):
    dt = T/N
    u, m, d, pu, pm, pd = tri_probs_moment_match(dt, r, q, sigma)
    disc = np.exp(-r*dt)

    # Terminal values after N steps, k in [-N..N]
    # S(N,k) = S0 * u^{max(k,0)} * d^{max(-k,0)}  with m=1 in between.
    ks = np.arange(-N, N+1)
    ST = S0 * (u**np.maximum(ks,0)) * (d**np.maximum(-ks,0))
    V = np.maximum(0.0, ST-K) if option=='call' else np.maximum(0.0, K-ST)

    # Backward induction
    for i in range(N-1, -1, -1):
        ks = np.arange(-i, i+1)
        # map next layer indices: for node k at time i, children at time i+1 are k+1 (up), k (mid), k-1 (down)
        V_up   = V[(ks+1) + (i)]     # index shift: next layer length = 2(i+1)+1; center at index i+1.
        V_mid  = V[(ks)   + (i)]
        V_down = V[(ks-1) + (i)]
        V = disc * (pu*V_up + pm*V_mid + pd*V_down)
        if american:
            S_i = S0 * (u**np.maximum(ks,0)) * (d**np.maximum(-ks,0))
            exercise = np.maximum(0.0, S_i-K) if option=='call' else np.maximum(0.0, K-S_i)
            V = np.maximum(V, exercise)
    return float(V[0])

# Trinomial Greeks via bumping

def trinomial_greeks(S0, K, T, r, q, sigma, N=200, option='call', american=False, dS=0.01, dT=1/252, dV=0.0001, dr=0.0001):
    P0 = trinomial_price(S0, K, T, r, q, sigma, N, option, american)
    P_up = trinomial_price(S0+dS, K, T, r, q, sigma, N, option, american)
    P_dn = trinomial_price(S0-dS, K, T, r, q, sigma, N, option, american)
    delta = (P_up - P_dn)/(2*dS)
    gamma = (P_up - 2*P0 + P_dn)/(dS**2)
    T_dn = max(1e-6, T - dT)
    P_Tdn = trinomial_price(S0, K, T_dn, r, q, sigma, N, option, american)
    theta = (P_Tdn - P0) / (-dT)
    P_sv_up = trinomial_price(S0, K, T, r, q, sigma+dV, N, option, american)
    P_sv_dn = trinomial_price(S0, K, T, r, q, sigma-dV, N, option, american)
    vega = (P_sv_up - P_sv_dn)/(2*dV)
    P_r_up = trinomial_price(S0, K, T, r+dr, q, sigma, N, option, american)
    P_r_dn = trinomial_price(S0, K, T, r-dr, q, sigma, N, option, american)
    rho = (P_r_up - P_r_dn)/(2*dr)
    return {'price': P0, 'delta': delta, 'gamma': gamma, 'theta': theta, 'vega': vega, 'rho': rho}

# --- LSMC for American options ---

def lsmc_american_price(S0, K, T, r, q, sigma, option='put', n_paths=50000, n_steps=60, seed=1234):
    rng = np.random.default_rng(seed)
    dt = T/n_steps
    disc = np.exp(-r*dt)
    drift = (r - q - 0.5*sigma**2)*dt
    vol = sigma*np.sqrt(dt)

    Z = rng.standard_normal((n_paths, n_steps))
    S = np.zeros((n_paths, n_steps+1))
    S[:,0] = S0
    for t in range(1, n_steps+1):
        S[:,t] = S[:,t-1]*np.exp(drift + vol*Z[:,t-1])

    if option=='call':
        intrinsic = np.maximum(S - K, 0.0)
    else:
        intrinsic = np.maximum(K - S, 0.0)

    # Cashflow and exercise time arrays
    CF = intrinsic[:,-1]
    tau = np.full(n_paths, n_steps, dtype=int)

    # Basis functions: [1, S, S^2]
    for t in range(n_steps-1, 0, -1):
        itm = intrinsic[:,t] > 1e-12
        X = S[itm, t]
        if X.size == 0:
            continue
        Y = CF[itm]* (disc**(tau[itm]-t))  # discount back to time t
        A = np.vstack([np.ones_like(X), X, X*X]).T
        # Linear regression for continuation value
        coeff, *_ = np.linalg.lstsq(A, Y, rcond=None)
        C_hat = (A @ coeff)
        ex_val = intrinsic[itm, t]
        exercise = ex_val > C_hat
        # Update CF and tau where exercise
        idx = np.where(itm)[0][exercise]
        CF[idx] = ex_val[exercise]
        tau[idx] = t

    price = np.mean(CF * (disc**tau))
    stderr = np.std(CF * (disc**tau), ddof=1)/np.sqrt(n_paths)
    return {'price': float(price), 'stderr': float(stderr)}

# --- Hedging P&L distribution (European call, ATM 60d) ---

def hedge_pnl_distribution(S0, K, T, r, q, sigma, n_paths=2000, n_steps=60, rebalance_per_day=1, seed=7):
    rng = np.random.default_rng(seed)
    dt = T/n_steps
    drift = (r - q - 0.5*sigma**2)*dt
    vol = sigma*np.sqrt(dt)

    pnl = np.zeros(n_paths)
    for p in range(n_paths):
        z = rng.standard_normal(n_steps)
        S = np.empty(n_steps+1)
        S[0] = S0
        for t in range(1, n_steps+1):
            S[t] = S[t-1]*np.exp(drift + vol*z[t-1])
        # Delta-hedge with BS delta daily
        cash = 0.0
        delta_prev = 0.0
        for t in range(n_steps):
            tau = T - t*dt
            d1 = (np.log(S[t]/K) + (r - q + 0.5*sigma**2)*tau)/(sigma*np.sqrt(tau)) if tau>0 else 0.0
            delta = np.exp(-q*tau)*norm.cdf(d1)
            d_shares = delta - delta_prev
            cash *= np.exp(r*dt)
            cash -= d_shares * S[t]
            delta_prev = delta
        payoff = max(0.0, S[-1]-K)
        cash *= np.exp(r*dt)
        replication = delta_prev*S[-1] + cash
        pnl[p] = replication - payoff
    stats = {
        'mean': float(np.mean(pnl)),
        'std': float(np.std(pnl, ddof=1)),
        'p05': float(np.percentile(pnl, 5)),
        'p50': float(np.percentile(pnl, 50)),
        'p95': float(np.percentile(pnl, 95))
    }
    # Plot histogram
    plt.figure(figsize=(7.5,4))
    plt.hist(pnl, bins=50, alpha=0.75, color='steelblue', edgecolor='white')
    plt.title('Delta-Hedging P&L Distribution (European Call, ATM, 60d)')
    plt.xlabel('Replication P&L (Replication − Payoff)')
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.savefig('hedge_pnl_hist.png', dpi=150)
    plt.close()
    return pnl, stats

# --- Implied vol inversion (synthetic prices) ---

def implied_vol_bs(price, S, K, T, r, q, option='call', tol=1e-7, max_iter=100):
    # Brent/bi-section hybrid
    lo, hi = 1e-6, 3.0
    for _ in range(max_iter):
        mid = 0.5*(lo+hi)
        f_mid = bs_price(S,K,T,r,q,mid,option) - price
        f_lo = bs_price(S,K,T,r,q,lo,option) - price
        if f_mid==0 or (hi-lo)<tol:
            return float(mid)
        if np.sign(f_mid)==np.sign(f_lo):
            lo = mid
        else:
            hi = mid
    return float(mid)

# -------------------- Run tasks --------------------
rows = []
terms_days = [30,60,90]
for sym, info in market.items():
    S0 = info['S0']; sigma = info['sigma']
    for days in terms_days:
        T = days/365
        K = round(S0,2)
        N = max(200, int(200*sqrt(days/30)))
        for opt in ['call','put']:
            tri_eu = trinomial_greeks(S0,K,T,r,q,sigma,N,opt,american=False)
            tri_am = trinomial_greeks(S0,K,T,r,q,sigma,N,opt,american=True)
            # For reference add BS price for European
            bs_eu = bs_price(S0,K,T,r,q,sigma,opt)
            rows.append({
                'Symbol': sym,
                'Days': days,
                'Option': opt,
                'S0': S0,
                'Strike': K,
                'Sigma': sigma,
                'TRI_Euro_Price': tri_eu['price'],
                'TRI_Euro_Delta': tri_eu['delta'],
                'TRI_Euro_Gamma': tri_eu['gamma'],
                'TRI_Euro_Theta/yr': tri_eu['theta'],
                'TRI_Euro_Vega': tri_eu['vega'],
                'TRI_Euro_Rho': tri_eu['rho'],
                'TRI_Amer_Price': tri_am['price'],
                'TRI_Amer_Delta': tri_am['delta'],
                'TRI_Amer_Gamma': tri_am['gamma'],
                'TRI_Amer_Theta/yr': tri_am['theta'],
                'TRI_Amer_Vega': tri_am['vega'],
                'TRI_Amer_Rho': tri_am['rho'],
                'BS_Euro_Price': bs_eu
            })

tri_df = pd.DataFrame(rows)
tri_df.to_csv('trinomial_fixed_outputs.csv', index=False)

# LSMC prices (ATM, 60d) for both symbols, put & call
lsmc_rows = []
for sym, info in market.items():
    S0 = info['S0']; sigma = info['sigma']
    T = 60/365; K = round(S0,2)
    for opt in ['call','put']:
        res = lsmc_american_price(S0,K,T,r,q,sigma,option=opt,n_paths=40000,n_steps=60,seed=777)
        lsmc_rows.append({'Symbol': sym,'Option': opt,'T_days':60,'Strike':K,'Price':res['price'],'StdErr':res['stderr']})
lsmc_df = pd.DataFrame(lsmc_rows)
lsmc_df.to_csv('lsmc_american_60d.csv', index=False)

# Hedging P&L for both symbols (European ATM call 60d)
hedge_stats = {}
for sym, info in market.items():
    S0 = info['S0']; sigma = info['sigma']
    T = 60/365; K = round(S0,2)
    pnl, stats = hedge_pnl_distribution(S0,K,T,r,q,sigma,n_paths=3000,n_steps=60,seed=123+len(sym))
    hedge_stats[sym] = stats

# Save hedge stats
hedge_stats_df = pd.DataFrame([{'Symbol':k, **v} for k,v in hedge_stats.items()])
hedge_stats_df.to_csv('hedge_pnl_stats.csv', index=False)

# Implied vol from synthetic market prices (clearly labeled synthetic)
impl_rows = []
for sym, info in market.items():
    S0 = info['S0']; sigma = info['sigma']
    K = round(S0,2)
    for days in [30,60]:
        T = days/365
        # Synthetic market price: 110% of BS price to mimic a slight vol premium
        for opt in ['call','put']:
            bs_p = bs_price(S0,K,T,r,q,sigma,opt)
            mkt = 1.10*bs_p
            iv = implied_vol_bs(mkt, S0, K, T, r, q, option=opt)
            impl_rows.append({'Symbol':sym,'Days':days,'Option':opt,'S0':S0,'Strike':K,'SyntheticMktPrice':mkt,'ImpliedVol':iv,'BS_ref_with_realized_sigma':bs_p})
impl_df = pd.DataFrame(impl_rows)
impl_df.to_csv('implied_vol_synthetic.csv', index=False)

# Return quick previews for chat
tri_preview = tri_df.head(6).to_dict(orient='records')
lsmc_preview = lsmc_df.to_dict(orient='records')
hedge_preview = hedge_stats_df.to_dict(orient='records')
impl_preview = impl_df.head(8).to_dict(orient='records')

tri_preview, lsmc_preview, hedge_preview, impl_preview


([{'Symbol': 'SEA',
   'Days': 30,
   'Option': 'call',
   'S0': 16.0,
   'Strike': 16.0,
   'Sigma': 0.2029992290715125,
   'TRI_Euro_Price': 0.421553447907216,
   'TRI_Euro_Delta': 0.057291105203691184,
   'TRI_Euro_Gamma': 0.03245273125507264,
   'TRI_Euro_Theta/yr': 3.894688999521907,
   'TRI_Euro_Vega': 14.134482532154879,
   'TRI_Euro_Rho': -31.56506451792995,
   'TRI_Amer_Price': 2.0189952002571134,
   'TRI_Amer_Delta': 0.5359288680842234,
   'TRI_Amer_Gamma': 0.006032154526991462,
   'TRI_Amer_Theta/yr': 16.48813069739074,
   'TRI_Amer_Vega': 44.43915072885884,
   'TRI_Amer_Rho': -89.51559549646103,
   'BS_Euro_Price': 0.40440140615699605},
  {'Symbol': 'SEA',
   'Days': 30,
   'Option': 'put',
   'S0': 16.0,
   'Strike': 16.0,
   'Sigma': 0.2029992290715125,
   'TRI_Euro_Price': 10.363699338512733,
   'TRI_Euro_Delta': -0.3172236203976553,
   'TRI_Euro_Gamma': 0.032452731311138905,
   'TRI_Euro_Theta/yr': 33.49345122463333,
   'TRI_Euro_Vega': 16.82864925606431,
   'TRI_Euro_R

In [7]:
# --- Globals / inputs ---
r = 0.05
q = 0.0
rng = default_rng(20260208)

# Helper to prep and compute latest S0 and YZ vol from raw files
TRADING_DAYS = 252

def prep_df(df):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date').reset_index(drop=True)
    if 'Adj Close' in df.columns:
        df['AdjClose'] = pd.to_numeric(df['Adj Close'], errors='coerce')
    else:
        df['AdjClose'] = pd.to_numeric(df['Close'], errors='coerce')
    for col in ['Open','High','Low','Close']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    df['logret'] = np.log(df['AdjClose']).diff()
    df['overnight'] = np.log(df['Open'] / df['AdjClose'].shift(1))
    df['oc'] = np.log(df['Close'] / df['Open'])
    df['hl'] = np.log(df['High'] / df['Low'])
    df['rs'] = np.log(df['High']/df['Open']) * np.log(df['High']/df['Close']) + \
               np.log(df['Low']/df['Open'])  * np.log(df['Low']/df['Close'])
    return df

def yang_zhang_vol(df):
    x = df.dropna(subset=['overnight','oc','rs'])
    n = len(x)
    if n < 2:
        return np.nan
    k = 0.34 / (1.34 + (n + 1) / (n - 1))
    var_overnight = x['overnight'].var(ddof=1)
    var_oc = x['oc'].var(ddof=1)
    rs_mean = x['rs'].mean()
    var = var_overnight + k*var_oc + (1 - k)*rs_mean
    return np.sqrt(TRADING_DAYS * var)

# Load raw files
raw_files = {
    'SEA': 'SEA.csv',
    'AMKBY': 'AMKBY.csv'
}

market = {}
for s,f in raw_files.items():
    df = pd.read_csv(f)
    df = prep_df(df)
    # compute rolling 21d YZ and take last
    # simple rolling function
    def rolling_yz(df, window=21):
        out=[]
        for i in range(len(df)):
            lo=max(0,i-window+1)
            out.append(yang_zhang_vol(df.iloc[lo:i+1]))
        return pd.Series(out,index=df.index)
    df['YZ_21d'] = rolling_yz(df,21)
    S0 = float(df['AdjClose'].iloc[-1])
    sigma = float(pd.to_numeric(df['YZ_21d'], errors='coerce').dropna().iloc[-1])
    market[s] = {'S0': S0, 'sigma': sigma}

# --- Black-Scholes for reference ---
from scipy.stats import norm

def bs_price(S, K, T, r, q, sigma, option='call'):
    if T<=0 or sigma<=0:
        return max(0.0, (S-K) if option=='call' else (K-S))
    d1 = (np.log(S/K) + (r - q + 0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    if option=='call':
        return float(S*np.exp(-q*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2))
    else:
        return float(K*np.exp(-r*T)*norm.cdf(-d2) - S*np.exp(-q*T)*norm.cdf(-d1))

# --- Improved Trinomial Tree (moment-matching on S) ---

def tri_probs_moment_match(dt, r, q, sigma):
    u = np.exp(sigma*np.sqrt(dt))
    d = 1.0/u
    m1 = np.exp((r - q)*dt)
    m2 = np.exp(2*(r - q)*dt + sigma**2 * dt)
    A = np.array([[1.0, 1.0, 1.0],
                  [u,   1.0, d  ],
                  [u*u, 1.0, d*d]])
    b = np.array([1.0, m1, m2])
    pu, pm, pd = np.linalg.solve(A, b)
    probs = np.array([pu, pm, pd])
    if np.any(probs < -1e-10) or np.any(probs > 1+1e-10):
        dx = sigma*np.sqrt(3*dt)
        u = np.exp(dx); d = np.exp(-dx)
        nu = r - q - 0.5*sigma**2
        pu = 1/6 + (nu*np.sqrt(dt))/(2*dx) + (sigma**2*dt)/(2*dx*dx)
        pm = 2/3 - (sigma**2*dt)/(dx*dx)
        pd = 1 - pu - pm
        probs = np.clip(np.array([pu, pm, pd]), 0.0, 1.0)
        s = probs.sum()
        probs = probs/s if s>0 else np.array([1/4,1/2,1/4])
        return u, 1.0, d, probs[0], probs[1], probs[2]
    return u, 1.0, d, float(pu), float(pm), float(pd)


def trinomial_price(S0, K, T, r, q, sigma, N=200, option='call', american=False):
    dt = T/N
    u, m, d, pu, pm, pd = tri_probs_moment_match(dt, r, q, sigma)
    disc = np.exp(-r*dt)
    # terminal
    ks = np.arange(-N, N+1)
    ST = S0 * (u**np.maximum(ks,0)) * (d**np.maximum(-ks,0))
    V = np.maximum(0.0, ST-K) if option=='call' else np.maximum(0.0, K-ST)
    # backward
    for i in range(N-1, -1, -1):
        ks = np.arange(-i, i+1)
        V_up   = V[(ks+1)+(i)]
        V_mid  = V[(ks)+(i)]
        V_down = V[(ks-1)+(i)]
        V = np.exp(-r*dt)*(pu*V_up + pm*V_mid + pd*V_down)
        if american:
            S_i = S0 * (u**np.maximum(ks,0)) * (d**np.maximum(-ks,0))
            exercise = np.maximum(0.0, S_i-K) if option=='call' else np.maximum(0.0, K-S_i)
            V = np.maximum(V, exercise)
    return float(V[0])


def trinomial_greeks(S0, K, T, r, q, sigma, N=200, option='call', american=False, dS=0.01, dT=1/252, dV=0.0001, dr=0.0001):
    P0 = trinomial_price(S0, K, T, r, q, sigma, N, option, american)
    P_up = trinomial_price(S0+dS, K, T, r, q, sigma, N, option, american)
    P_dn = trinomial_price(S0-dS, K, T, r, q, sigma, N, option, american)
    delta = (P_up - P_dn)/(2*dS)
    gamma = (P_up - 2*P0 + P_dn)/(dS**2)
    T_dn = max(1e-6, T - dT)
    P_Tdn = trinomial_price(S0, K, T_dn, r, q, sigma, N, option, american)
    theta = (P_Tdn - P0) / (-dT)
    P_sv_up = trinomial_price(S0, K, T, r, q, sigma+dV, N, option, american)
    P_sv_dn = trinomial_price(S0, K, T, r, q, sigma-dV, N, option, american)
    vega = (P_sv_up - P_sv_dn)/(2*dV)
    P_r_up = trinomial_price(S0, K, T, r+dr, q, sigma, N, option, american)
    P_r_dn = trinomial_price(S0, K, T, r-dr, q, sigma, N, option, american)
    rho = (P_r_up - P_r_dn)/(2*dr)
    return {'price': P0, 'delta': delta, 'gamma': gamma, 'theta': theta, 'vega': vega, 'rho': rho}

# LSMC

def lsmc_american_price(S0, K, T, r, q, sigma, option='put', n_paths=30000, n_steps=60, seed=777):
    rng = np.random.default_rng(seed)
    dt = T/n_steps
    disc = np.exp(-r*dt)
    drift = (r - q - 0.5*sigma**2)*dt
    vol = sigma*np.sqrt(dt)
    Z = rng.standard_normal((n_paths, n_steps))
    S = np.zeros((n_paths, n_steps+1))
    S[:,0] = S0
    for t in range(1, n_steps+1):
        S[:,t] = S[:,t-1] * np.exp(drift + vol*Z[:,t-1])
    if option=='call':
        intrinsic = np.maximum(S - K, 0.0)
    else:
        intrinsic = np.maximum(K - S, 0.0)
    CF = intrinsic[:,-1]
    tau = np.full(n_paths, n_steps, dtype=int)
    for t in range(n_steps-1, 0, -1):
        itm = intrinsic[:,t] > 1e-12
        X = S[itm, t]
        if X.size==0:
            continue
        Y = CF[itm] * (disc**(tau[itm]-t))
        A = np.vstack([np.ones_like(X), X, X*X]).T
        coeff, *_ = np.linalg.lstsq(A, Y, rcond=None)
        C_hat = (A @ coeff)
        ex_val = intrinsic[itm, t]
        exercise = ex_val > C_hat
        idx = np.where(itm)[0][exercise]
        CF[idx] = ex_val[exercise]
        tau[idx] = t
    price = np.mean(CF * (disc**tau))
    stderr = np.std(CF * (disc**tau), ddof=1)/np.sqrt(n_paths)
    return {'price': float(price), 'stderr': float(stderr)}

# Hedging P&L

def hedge_pnl_distribution(S0, K, T, r, q, sigma, n_paths=3000, n_steps=60, seed=42):
    rng = np.random.default_rng(seed)
    dt = T/n_steps
    drift = (r - q - 0.5*sigma**2)*dt
    vol = sigma*np.sqrt(dt)
    pnl = np.zeros(n_paths)
    for p in range(n_paths):
        z = rng.standard_normal(n_steps)
        S = np.empty(n_steps+1)
        S[0] = S0
        for t in range(1, n_steps+1):
            S[t] = S[t-1]*np.exp(drift + vol*z[t-1])
        cash = 0.0
        delta_prev = 0.0
        for t in range(n_steps):
            tau = T - t*dt
            if tau<=0:
                delta = 0.0
            else:
                d1 = (np.log(S[t]/K) + (r - q + 0.5*sigma**2)*tau)/(sigma*np.sqrt(tau))
                delta = np.exp(-q*tau)*norm.cdf(d1)
            d_shares = delta - delta_prev
            cash *= np.exp(r*dt)
            cash -= d_shares * S[t]
            delta_prev = delta
        payoff = max(0.0, S[-1]-K)
        cash *= np.exp(r*dt)
        replication = delta_prev*S[-1] + cash
        pnl[p] = replication - payoff
    stats = {
        'mean': float(np.mean(pnl)),
        'std': float(np.std(pnl, ddof=1)),
        'p05': float(np.percentile(pnl, 5)),
        'p50': float(np.percentile(pnl, 50)),
        'p95': float(np.percentile(pnl, 95))
    }
    plt.figure(figsize=(7.5,4))
    plt.hist(pnl, bins=50, alpha=0.8, color='steelblue', edgecolor='white')
    plt.title('Delta-Hedging P&L Distribution (European Call, ATM, 60d)')
    plt.xlabel('Replication P&L (Replication − Payoff)')
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.savefig('hedge_pnl_hist.png', dpi=150)
    plt.close()
    return pnl, stats

# Implied vol

def implied_vol_bs(price, S, K, T, r, q, option='call', tol=1e-7, max_iter=100):
    lo, hi = 1e-6, 3.0
    for _ in range(max_iter):
        mid = 0.5*(lo+hi)
        f_mid = bs_price(S,K,T,r,q,mid,option) - price
        f_lo = bs_price(S,K,T,r,q,lo,option) - price
        if f_mid==0 or (hi-lo)<tol:
            return float(mid)
        if np.sign(f_mid)==np.sign(f_lo):
            lo = mid
        else:
            hi = mid
    return float(mid)

# ---------------- Execute ----------------
# 1) Trinomial prices & Greeks (Euro & American)
rows = []
for sym, info in market.items():
    S0 = info['S0']; sigma = info['sigma']
    for days in [30,60,90]:
        T = days/365
        K = round(S0,2)
        N = max(200, int(250*sqrt(days/30)))
        for opt in ['call','put']:
            tri_eu = trinomial_greeks(S0,K,T,r,q,sigma,N,opt,american=False)
            tri_am = trinomial_greeks(S0,K,T,r,q,sigma,N,opt,american=True)
            bs_eu = bs_price(S0,K,T,r,q,sigma,opt)
            rows.append({'Symbol': sym,'Days':days,'Option':opt,'S0':S0,'K':K,'Sigma':sigma,
                         'TRI_Euro_Price':tri_eu['price'],'TRI_Euro_Delta':tri_eu['delta'],'TRI_Euro_Gamma':tri_eu['gamma'],'TRI_Euro_Theta/yr':tri_eu['theta'],'TRI_Euro_Vega':tri_eu['vega'],'TRI_Euro_Rho':tri_eu['rho'],
                         'TRI_Amer_Price':tri_am['price'],'TRI_Amer_Delta':tri_am['delta'],'TRI_Amer_Gamma':tri_am['gamma'],'TRI_Amer_Theta/yr':tri_am['theta'],'TRI_Amer_Vega':tri_am['vega'],'TRI_Amer_Rho':tri_am['rho'],
                         'BS_Euro_Price':bs_eu})
tri_df = pd.DataFrame(rows)
tri_df.to_csv('trinomial_fixed_outputs.csv', index=False)

# 2) LSMC American 60d
lsmc_rows = []
for sym, info in market.items():
    S0 = info['S0']; sigma = info['sigma']
    T = 60/365; K = round(S0,2)
    for opt in ['call','put']:
        res = lsmc_american_price(S0,K,T,r,q,sigma,option=opt,n_paths=30000,n_steps=60,seed=2026)
        lsmc_rows.append({'Symbol':sym,'Option':opt,'T_days':60,'Strike':K,'Price':res['price'],'StdErr':res['stderr']})
lsmc_df = pd.DataFrame(lsmc_rows)
lsmc_df.to_csv('lsmc_american_60d.csv', index=False)

# 3) Hedging P&L distribution (European ATM 60d)
hedge_stats = []
for sym, info in market.items():
    S0 = info['S0']; sigma = info['sigma']
    T = 60/365; K = round(S0,2)
    pnl, stats = hedge_pnl_distribution(S0,K,T,r,q,sigma,n_paths=4000,n_steps=60,seed=len(sym)+11)
    hedge_stats.append({'Symbol':sym, **stats})
hedge_stats_df = pd.DataFrame(hedge_stats)
hedge_stats_df.to_csv('hedge_pnl_stats.csv', index=False)

# 4) Synthetic implied vols
impl_rows = []
for sym, info in market.items():
    S0 = info['S0']; sigma = info['sigma']
    K = round(S0,2)
    for days in [30,60]:
        T = days/365
        for opt in ['call','put']:
            bs_p = bs_price(S0,K,T,r,q,sigma,opt)
            mkt = 1.10*bs_p  # synthetic premium 10%
            iv = implied_vol_bs(mkt, S0, K, T, r, q, option=opt)
            impl_rows.append({'Symbol':sym,'Days':days,'Option':opt,'S0':S0,'Strike':K,'SyntheticMktPrice':mkt,'ImpliedVol':iv,'BS_ref_price_using_realized_sigma':bs_p})
impl_df = pd.DataFrame(impl_rows)
impl_df.to_csv('implied_vol_synthetic.csv', index=False)

# Return quick previews
tri_preview = tri_df.head(8).to_dict(orient='records')
lsmc_preview = lsmc_df.to_dict(orient='records')
hedge_preview = hedge_stats_df.to_dict(orient='records')
impl_preview = impl_df.head(8).to_dict(orient='records')

tri_preview, lsmc_preview, hedge_preview, impl_preview

([{'Symbol': 'SEA',
   'Days': 30,
   'Option': 'call',
   'S0': 16.0,
   'K': 16.0,
   'Sigma': 0.20299922907151258,
   'TRI_Euro_Price': 0.3713633545377499,
   'TRI_Euro_Delta': 0.04703623231385734,
   'TRI_Euro_Gamma': 0.021113537091999035,
   'TRI_Euro_Theta/yr': 3.5808170131109036,
   'TRI_Euro_Vega': 14.549992441977576,
   'TRI_Euro_Rho': -33.48009250494804,
   'TRI_Amer_Price': 1.978014669152148,
   'TRI_Amer_Delta': 0.5190499262289983,
   'TRI_Amer_Gamma': 0.016664981665748257,
   'TRI_Amer_Theta/yr': 16.476775476922622,
   'TRI_Amer_Vega': 48.327101717847704,
   'TRI_Amer_Rho': -100.69536641519328,
   'BS_Euro_Price': 0.40440140615699605},
  {'Symbol': 'SEA',
   'Days': 30,
   'Option': 'put',
   'S0': 16.0,
   'K': 16.0,
   'Sigma': 0.20299922907151258,
   'TRI_Euro_Price': 11.15567689172366,
   'TRI_Euro_Delta': -0.2748430153761028,
   'TRI_Euro_Gamma': 0.021113536927686027,
   'TRI_Euro_Theta/yr': 32.92023353991774,
   'TRI_Euro_Vega': 16.304484323006463,
   'TRI_Euro_Rho':

In [9]:
# Reuse market dict by recomputing quickly
TRADING_DAYS=252

def prep_df(df):
    df=df.copy(); df['Date']=pd.to_datetime(df['Date']); df=df.sort_values('Date')
    df['AdjClose']=pd.to_numeric(df.get('Adj Close', df['Close']), errors='coerce')
    for c in ['Open','High','Low','Close']: df[c]=pd.to_numeric(df[c], errors='coerce')
    df['logret']=np.log(df['AdjClose']).diff()
    df['overnight']=np.log(df['Open']/df['AdjClose'].shift(1))
    df['oc']=np.log(df['Close']/df['Open'])
    df['rs']=np.log(df['High']/df['Open'])*np.log(df['High']/df['Close'])+np.log(df['Low']/df['Open'])*np.log(df['Low']/df['Close'])
    return df

def yz(df):
    x=df.dropna(subset=['overnight','oc','rs']); n=len(x)
    if n<2: return np.nan
    k=0.34/(1.34+(n+1)/(n-1))
    var=x['overnight'].var(ddof=1)+k*x['oc'].var(ddof=1)+(1-k)*x['rs'].mean()
    return np.sqrt(TRADING_DAYS*var)

files={'SEA':'SEA.csv','AMKBY':'AMKBY.csv'}
market={}
for s,f in files.items():
    df=pd.read_csv(f); df=prep_df(df)
    # rolling 21d YZ
    vol=[]
    for i in range(len(df)):
        lo=max(0,i-21+1); vol.append(yz(df.iloc[lo:i+1]))
    df['YZ_21d']=vol
    market[s]={'S0':float(df['AdjClose'].iloc[-1]),'sigma':float(pd.to_numeric(df['YZ_21d']).dropna().iloc[-1])}

r=0.05; q=0.0

def bs_price(S,K,T,r,q,sigma,option='call'):
    if T<=0 or sigma<=0: return max(0.0, (S-K) if option=='call' else (K-S))
    d1=(np.log(S/K)+(r-q+0.5*sigma**2)*T)/(sigma*np.sqrt(T)); d2=d1-sigma*np.sqrt(T)
    if option=='call': return float(S*np.exp(-q*T)*norm.cdf(d1)-K*np.exp(-r*T)*norm.cdf(d2))
    else: return float(K*np.exp(-r*T)*norm.cdf(-d2)-S*np.exp(-q*T)*norm.cdf(-d1))

def tri_probs(dt,r,q,sigma):
    u=np.exp(sigma*np.sqrt(dt)); d=1/u
    m1=np.exp((r-q)*dt); m2=np.exp(2*(r-q)*dt+sigma**2*dt)
    A=np.array([[1.0,1.0,1.0],[u,1.0,d],[u*u,1.0,d*d]]); b=np.array([1.0,m1,m2])
    pu,pm,pd=np.linalg.solve(A,b)
    probs=np.array([pu,pm,pd])
    if np.any(probs<-1e-10) or np.any(probs>1+1e-10):
        dx=sigma*np.sqrt(3*dt); u=np.exp(dx); d=np.exp(-dx)
        nu=r-q-0.5*sigma**2
        pu=1/6+(nu*np.sqrt(dt))/(2*dx)+(sigma**2*dt)/(2*dx*dx)
        pm=2/3-(sigma**2*dt)/(dx*dx); pd=1-pu-pm
        probs=np.clip(np.array([pu,pm,pd]),0,1); s=probs.sum(); probs=probs/s if s>0 else np.array([1/4,1/2,1/4])
        return u,1.0,d,probs[0],probs[1],probs[2]
    return u,1.0,d,float(pu),float(pm),float(pd)


def tri_price(S0,K,T,r,q,sigma,N=300,option='call',american=False):
    dt=T/N; u,m,d,pu,pm,pd=tri_probs(dt,r,q,sigma); disc=np.exp(-r*dt)
    ks=np.arange(-N,N+1)
    ST=S0*(u**np.maximum(ks,0))*(d**np.maximum(-ks,0))
    V=np.maximum(0.0,ST-K) if option=='call' else np.maximum(0.0,K-ST)
    for i in range(N-1,-1,-1):
        ks=np.arange(-i,i+1)
        # Correct indexing: next layer center is (i+1)
        V_up=V[(ks+1)+(i+1)]
        V_mid=V[(ks)+(i+1)]
        V_down=V[(ks-1)+(i+1)]
        V=disc*(pu*V_up+pm*V_mid+pd*V_down)
        if american:
            S_i=S0*(u**np.maximum(ks,0))*(d**np.maximum(-ks,0))
            exercise=np.maximum(0.0,S_i-K) if option=='call' else np.maximum(0.0,K-S_i)
            V=np.maximum(V,exercise)
    return float(V[0])

# quick comparison: SEA 30d ATM call
S0=market['SEA']['S0']; sigma=market['SEA']['sigma']; T=30/365; K=round(S0,2)
bs=bs_price(S0,K,T,r,q,sigma,'call')
tri_e=tri_price(S0,K,T,r,q,sigma,option='call',american=False)
tri_a=tri_price(S0,K,T,r,q,sigma,option='call',american=True)
bs, tri_e, tri_a

(0.40440140615699605, 2.5851832547837144, 2.5851832547837144)

In [12]:
# Reuse prior helper functions

def bs_price(S,K,T,r,q,sigma,option='call'):
    if T<=0 or sigma<=0: return max(0.0, (S-K) if option=='call' else (K-S))
    d1=(np.log(S/K)+(r-q+0.5*sigma**2)*T)/(sigma*np.sqrt(T)); d2=d1-sigma*np.sqrt(T)
    if option=='call': return float(S*np.exp(-q*T)*norm.cdf(d1)-K*np.exp(-r*T)*norm.cdf(d2))
    else: return float(K*np.exp(-r*T)*norm.cdf(-d2)-S*np.exp(-q*T)*norm.cdf(-d1))


def kr_params(dt,r,q,sigma,lam=1.0):
    dx = lam*sigma*np.sqrt(3*dt)
    u = np.exp(dx); d = np.exp(-dx)
    nu = r - q - 0.5*sigma**2
    pu = 1/6 + (nu*np.sqrt(dt))/(2*dx) + (sigma**2*dt)/(2*dx*dx)
    pm = 2/3 - (sigma**2*dt)/(dx*dx)
    pd = 1 - pu - pm
    probs = np.clip(np.array([pu,pm,pd]), 0.0, 1.0)
    s = probs.sum(); probs = probs/s if s>0 else np.array([1/4,1/2,1/4])
    return u, 1.0, d, probs[0], probs[1], probs[2]


def tri_price(S0,K,T,r,q,sigma,N=50,option='call',american=False):
    dt=T/N
    u,m,d,pu,pm,pd=kr_params(dt,r,q,sigma,lam=1.0)
    disc=np.exp(-r*dt)
    ks=np.arange(-N,N+1)
    ST=S0*(u**np.maximum(ks,0))*(d**np.maximum(-ks,0))
    V=np.maximum(0.0,ST-K) if option=='call' else np.maximum(0.0,K-ST)
    for i in range(N-1,-1,-1):
        ks=np.arange(-i,i+1)
        V_up=V[(ks+1)+(i+1)]
        V_mid=V[(ks)+(i+1)]
        V_down=V[(ks-1)+(i+1)]
        V=disc*(pu*V_up+pm*V_mid+pd*V_down)
        if american:
            S_i=S0*(u**np.maximum(ks,0))*(d**np.maximum(-ks,0))
            exercise=np.maximum(0.0,S_i-K) if option=='call' else np.maximum(0.0,K-S_i)
            V=np.maximum(V,exercise)
    return float(V[0])

S0=16.0; sigma=0.203; r=0.05; q=0.0; T=30/365; K=16.0
for N in [25,50,100,200]:
    tri=tri_price(S0,K,T,r,q,sigma,N=N,option='call',american=False)
    print(N, tri)
print('BS', bs_price(S0,K,T,r,q,sigma,'call'))


25 0.9828429724782011
50 1.206196795865435
100 1.55870935560877
200 2.118353939211309
BS 0.4044028099378689


In [13]:
r=0.05; q=0.0; sigma=0.203; T=30/365; N=100; dt=T/N
lam=1.0
dx=lam*sigma*np.sqrt(3*dt)
u=np.exp(dx); d=np.exp(-dx)
nu_drift=r-q-0.5*sigma**2
pu=1/6 + (nu_drift*np.sqrt(dt))/(2*dx) + (sigma**2*dt)/(2*dx*dx)
pm=2/3 - (sigma**2*dt)/(dx*dx)
pd=1-pu-pm
pu, pm, pd, u, d


(np.float64(0.37513505707216),
 np.float64(0.3333333333333334),
 np.float64(0.2915316095945066),
 np.float64(1.0101312164816167),
 np.float64(0.9899703956116664))

In [15]:
# Inputs (reuse earlier)
r=0.05; q=0.0

# Market from earlier step
files={'SEA':'SEA.csv','AMKBY':'AMKBY.csv'}
TRADING_DAYS=252

def prep_df(df):
    df=df.copy(); df['Date']=pd.to_datetime(df['Date']); df=df.sort_values('Date')
    df['AdjClose']=pd.to_numeric(df.get('Adj Close', df['Close']), errors='coerce')
    for c in ['Open','High','Low','Close']: df[c]=pd.to_numeric(df[c], errors='coerce')
    df['logret']=np.log(df['AdjClose']).diff()
    df['overnight']=np.log(df['Open']/df['AdjClose'].shift(1))
    df['oc']=np.log(df['Close']/df['Open'])
    df['rs']=np.log(df['High']/df['Open'])*np.log(df['High']/df['Close'])+np.log(df['Low']/df['Open'])*np.log(df['Low']/df['Close'])
    return df

def yz(df):
    x=df.dropna(subset=['overnight','oc','rs']); n=len(x)
    if n<2: return np.nan
    k=0.34/(1.34+(n+1)/(n-1))
    var=x['overnight'].var(ddof=1)+k*x['oc'].var(ddof=1)+(1-k)*x['rs'].mean()
    return np.sqrt(TRADING_DAYS*var)

market={}
for s,f in files.items():
    df=pd.read_csv(f); df=prep_df(df)
    vol=[]
    for i in range(len(df)):
        lo=max(0,i-21+1); out=df.iloc[lo:i+1]
        x=out.dropna(subset=['overnight','oc','rs'])
        n=len(x)
        if n<2: vol.append(np.nan)
        else:
            k=0.34/(1.34+(n+1)/(n-1))
            var=x['overnight'].var(ddof=1)+k*x['oc'].var(ddof=1)+(1-k)*x['rs'].mean()
            vol.append(np.sqrt(TRADING_DAYS*var))
    df['YZ_21d']=vol
    market[s]={'S0':float(df['AdjClose'].iloc[-1]),'sigma':float(pd.to_numeric(df['YZ_21d']).dropna().iloc[-1])}

from math import log, sqrt

def bs_price(S,K,T,r,q,sigma,option='call'):
    if T<=0 or sigma<=0: return max(0.0,(S-K) if option=='call' else (K-S))
    d1=(np.log(S/K)+(r-q+0.5*sigma**2)*T)/(sigma*np.sqrt(T)); d2=d1-sigma*np.sqrt(T)
    if option=='call': return float(S*np.exp(-q*T)*norm.cdf(d1)-K*np.exp(-r*T)*norm.cdf(d2))
    else: return float(K*np.exp(-r*T)*norm.cdf(-d2)-S*np.exp(-q*T)*norm.cdf(-d1))

# Hedging with premium

def hedge_pnl_with_premium(S0,K,T,r,q,sigma,n_paths=4000,n_steps=60,seed=99):
    rng=np.random.default_rng(seed)
    dt=T/n_steps
    drift=(r-q-0.5*sigma**2)*dt
    vol=sigma*np.sqrt(dt)
    premium=bs_price(S0,K,T,r,q,sigma,'call')
    pnl=np.zeros(n_paths)
    for p in range(n_paths):
        z=rng.standard_normal(n_steps)
        S=np.empty(n_steps+1); S[0]=S0
        for t in range(1,n_steps+1):
            S[t]=S[t-1]*np.exp(drift+vol*z[t-1])
        cash=premium  # receive premium at t0
        delta_prev=0.0
        for t in range(n_steps):
            tau=T - t*dt
            if tau<=0:
                delta=0.0
            else:
                d1=(np.log(S[t]/K)+(r-q+0.5*sigma**2)*tau)/(sigma*np.sqrt(tau))
                delta=np.exp(-q*tau)*norm.cdf(d1)
            d_shares=delta-delta_prev
            cash*=np.exp(r*dt)
            cash-=d_shares*S[t]
            delta_prev=delta
        payoff=max(0.0,S[-1]-K)
        cash*=np.exp(r*dt)
        replication=delta_prev*S[-1]+cash
        pnl[p]=replication - payoff
    stats={'mean':float(np.mean(pnl)),'std':float(np.std(pnl,ddof=1)),'p05':float(np.percentile(pnl,5)),'p50':float(np.percentile(pnl,50)),'p95':float(np.percentile(pnl,95)),'premium':premium}
    return pnl, stats

hedge_stats=[]
for sym,info in market.items():
    S0=info['S0']; sigma=info['sigma']; T=60/365; K=round(S0,2)
    pnl, stats = hedge_pnl_with_premium(S0,K,T,r,q,sigma,4000,60,seed=len(sym)+99)
    hedge_stats.append({'Symbol':sym, **stats})
    # plot hist
    plt.figure(figsize=(7.5,4))
    plt.hist(pnl, bins=50, alpha=0.8, color='seagreen', edgecolor='white')
    plt.title(f'Hedging P&L with Premium (European Call, ATM 60d) - {sym}')
    plt.xlabel('P&L (receive premium at t0, delta-hedge)')
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.savefig(f'hedge_pnl_with_premium_{sym}.png', dpi=150)
    plt.close()

hedge_stats_df=pd.DataFrame(hedge_stats)
hedge_stats_df.to_csv('hedge_pnl_with_premium_stats.csv', index=False)

hedge_stats_df.to_dict(orient='records')

[{'Symbol': 'SEA',
  'mean': -0.000977485578081836,
  'std': 0.05752922280194789,
  'p05': -0.09667613829771601,
  'p50': -0.0010943141883021645,
  'p95': 0.09137265184562242,
  'premium': 0.5911465544537453},
 {'Symbol': 'AMKBY',
  'mean': 0.00014903600441976372,
  'std': 0.10026776043969961,
  'p05': -0.16375180836290917,
  'p50': 0.002442153353986026,
  'p95': 0.1620891572804375,
  'premium': 0.9604144979356191}]

In [23]:
# -----------------------------
# Utilities and Inputs
# -----------------------------
r = 0.05
q = 0.0
TRADING_DAYS = 252

# Load market from raw CSVs and compute latest 21d YZ vols (recomputing to be safe)
files = {'SEA':'SEA.csv',
         'AMKBY':'AMKBY.csv'}

def prep_df(df):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    df['AdjClose'] = pd.to_numeric(df.get('Adj Close', df['Close']), errors='coerce')
    for c in ['Open','High','Low','Close']:
        df[c] = pd.to_numeric(df[c], errors='coerce')
    df['overnight'] = np.log(df['Open']/df['AdjClose'].shift(1))
    df['oc'] = np.log(df['Close']/df['Open'])
    df['rs'] = np.log(df['High']/df['Open'])*np.log(df['High']/df['Close']) + np.log(df['Low']/df['Open'])*np.log(df['Low']/df['Close'])
    return df

def yz_vol(df):
    x = df.dropna(subset=['overnight','oc','rs'])
    n = len(x)
    if n < 2:
        return np.nan
    k = 0.34/(1.34 + (n+1)/(n-1))
    var = x['overnight'].var(ddof=1) + k*x['oc'].var(ddof=1) + (1-k)*x['rs'].mean()
    return np.sqrt(TRADING_DAYS * var)

market = {}
for s, f in files.items():
    raw = pd.read_csv(f)
    df = prep_df(raw)
    # rolling 21d YZ
    yz_roll = []
    for i in range(len(df)):
        lo = max(0, i-21+1)
        yz_roll.append(yz_vol(df.iloc[lo:i+1]))
    df['YZ_21d'] = yz_roll
    S0 = float(df['AdjClose'].iloc[-1])
    sigma = float(pd.to_numeric(df['YZ_21d']).dropna().iloc[-1])
    market[s] = {'S0': S0, 'sigma': sigma}

# Black-Scholes reference

def bs_price(S, K, T, r, q, sigma, option='call'):
    if T <= 0 or sigma <= 0:
        return max(0.0, (S-K) if option=='call' else (K-S))
    d1 = (np.log(S/K) + (r - q + 0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    if option=='call':
        return float(S*np.exp(-q*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2))
    else:
        return float(K*np.exp(-r*T)*norm.cdf(-d2) - S*np.exp(-q*T)*norm.cdf(-d1))

# Log-space trinomial parameters

def logspace_trinomial_params(dt, r, q, sigma):
    mu = (r - q - 0.5*sigma**2)
    A = mu*dt
    dx = sigma*np.sqrt(3*dt)
    B = sigma**2*dt + A*A
    pu = 0.5*(A/dx + B/(dx*dx))
    pd = 0.5*(-A/dx + B/(dx*dx))
    pm = 1.0 - (B/(dx*dx))
    probs = np.array([pu, pm, pd])
    probs = np.clip(probs, 0.0, 1.0)
    s = probs.sum()
    if s == 0:
        probs = np.array([1/4, 1/2, 1/4])
    else:
        probs = probs/s
    return dx, probs[0], probs[1], probs[2]


def trinomial_log_price(S0, K, T, r, q, sigma, N=200, option='call', american=False):
    dt = T/N
    dx, pu, pm, pd = logspace_trinomial_params(dt, r, q, sigma)
    disc = np.exp(-r*dt)
    ks = np.arange(-N, N+1)
    ST = S0 * np.exp(ks*dx)
    V = np.maximum(0.0, ST-K) if option=='call' else np.maximum(0.0, K-ST)
    for i in range(N-1, -1, -1):
        ks = np.arange(-i, i+1)
        V_up   = V[(ks+1) + (i+1)]
        V_mid  = V[(ks)   + (i+1)]
        V_down = V[(ks-1) + (i+1)]
        V = disc * (pu*V_up + pm*V_mid + pd*V_down)
        if american:
            S_i = S0 * np.exp(ks*dx)
            exercise = np.maximum(0.0, S_i-K) if option=='call' else np.maximum(0.0, K-S_i)
            V = np.maximum(V, exercise)
    return float(V[0])

# Bump-and-reprice Greeks

def tri_log_greeks(S0, K, T, r, q, sigma, N=200, option='call', american=False, dS=0.01, dT=1/252, dV=0.0001, dr=0.0001):
    P0 = trinomial_log_price(S0, K, T, r, q, sigma, N, option, american)
    P_up = trinomial_log_price(S0+dS, K, T, r, q, sigma, N, option, american)
    P_dn = trinomial_log_price(S0-dS, K, T, r, q, sigma, N, option, american)
    delta = (P_up - P_dn)/(2*dS)
    gamma = (P_up - 2*P0 + P_dn)/(dS**2)
    T_dn = max(1e-6, T - dT)
    P_Tdn = trinomial_log_price(S0, K, T_dn, r, q, sigma, N, option, american)
    theta = (P_Tdn - P0)/(-dT)
    P_sv_up = trinomial_log_price(S0, K, T, r, q, sigma+dV, N, option, american)
    P_sv_dn = trinomial_log_price(S0, K, T, r, q, sigma-dV, N, option, american)
    vega = (P_sv_up - P_sv_dn)/(2*dV)
    P_r_up = trinomial_log_price(S0, K, T, r+dr, q, sigma, N, option, american)
    P_r_dn = trinomial_log_price(S0, K, T, r-dr, q, sigma, N, option, american)
    rho = (P_r_up - P_r_dn)/(2*dr)
    return {'price': P0, 'delta': delta, 'gamma': gamma, 'theta': theta, 'vega': vega, 'rho': rho}

# (A) N-sweep validation for SEA 30d call
S0 = market['SEA']['S0']; sigma_sea = market['SEA']['sigma']
K = round(S0, 2)
T = 30/365
bs_ref = bs_price(S0, K, T, r, q, sigma_sea, 'call')
Ns = [25, 50, 100, 200, 400, 800]
tri_prices = []
for N in Ns:
    p = trinomial_log_price(S0, K, T, r, q, sigma_sea, N=N, option='call', american=False)
    tri_prices.append(p)

n_sweep_df = pd.DataFrame({'N': Ns, 'Trinomial_LogSpace_Price': tri_prices})
n_sweep_df['BS_Ref'] = bs_ref
n_sweep_df['Abs_Error'] = (n_sweep_df['Trinomial_LogSpace_Price'] - bs_ref).abs()
n_sweep_df.to_csv('trinomial_logspace_n_sweep_sea_30d_call.csv', index=False)

plt.figure(figsize=(7.5,4))
plt.plot(Ns, n_sweep_df['Trinomial_LogSpace_Price'], marker='o', label='Trinomial (log-space)')
plt.axhline(bs_ref, color='k', linestyle='--', label=f'BS ref = {bs_ref:.6f}')
plt.xscale('log', base=2)
plt.xlabel('Number of steps N (log scale)')
plt.ylabel('Price')
plt.title('N-sweep Validation: SEA 30d ATM Call (Trinomial Log-space vs BS)')
plt.legend()
plt.tight_layout()
plt.savefig('trinomial_logspace_n_sweep_plot.png', dpi=150)
plt.close()

# (B) Hedging experiments
from numpy.random import default_rng


def bs_delta(S, K, tau, r, q, sigma, option='call'):
    if tau <= 0 or sigma <= 0:
        return 1.0 if (option=='call' and S>K) else (0.0 if option=='call' else (-1.0 if S<K else -0.5))
    d1 = (np.log(S/K) + (r - q + 0.5*sigma**2)*tau)/(sigma*np.sqrt(tau))
    if option=='call':
        return float(np.exp(-q*tau)*norm.cdf(d1))
    else:
        return float(-np.exp(-q*tau)*norm.cdf(-d1))


def hedge_pnl(S0, K, T, r, q, sigma_true, sigma_hedge, n_paths=4000, steps=60, seed=123, option='call'):
    rng = default_rng(seed)
    dt = T/steps
    drift = (r - q - 0.5*sigma_true**2)*dt
    vol = sigma_true*np.sqrt(dt)
    pnl = np.zeros(n_paths)
    premium = bs_price(S0, K, T, r, q, sigma_hedge, option)
    for p in range(n_paths):
        z = rng.standard_normal(steps)
        S = np.empty(steps+1)
        S[0] = S0
        for t in range(1, steps+1):
            S[t] = S[t-1]*np.exp(drift + vol*z[t-1])
        cash = premium
        delta_prev = 0.0
        for t in range(steps):
            tau = T - t*dt
            delta = bs_delta(S[t], K, tau, r, q, sigma_hedge, option)
            d_shares = delta - delta_prev
            cash *= np.exp(r*dt)
            cash -= d_shares * S[t]
            delta_prev = delta
        payoff = max(0.0, S[-1]-K) if option=='call' else max(0.0, K-S[-1])
        cash *= np.exp(r*dt)
        replication = delta_prev*S[-1] + cash
        pnl[p] = replication - payoff
    stats = {
        'mean': float(np.mean(pnl)),
        'std': float(np.std(pnl, ddof=1)),
        'p05': float(np.percentile(pnl, 5)),
        'p50': float(np.percentile(pnl, 50)),
        'p95': float(np.percentile(pnl, 95)),
        'premium_used': float(premium)
    }
    return pnl, stats

# Frequency test
hedge_rows = []
for sym, info in market.items():
    S0 = info['S0']; sigma = info['sigma']; K = round(S0,2)
    T = 60/365
    for steps in [60, 240]:
        pnl, stats = hedge_pnl(S0, K, T, r, q, sigma_true=sigma, sigma_hedge=sigma, n_paths=4000, steps=steps, seed=2026+steps)
        hedge_rows.append({'Symbol': sym, 'Steps': steps, **stats})
        plt.figure(figsize=(7.5,4))
        plt.hist(pnl, bins=60, alpha=0.8, color='slateblue', edgecolor='white')
        plt.title(f'Hedging P&L (freq test) {sym} — steps={steps}')
        plt.xlabel('P&L (premium included)')
        plt.ylabel('Frequency')
        plt.tight_layout()
        plt.savefig(f'hedge_freq_hist_{sym}_{steps}.png', dpi=150)
        plt.close()

hedge_freq_df = pd.DataFrame(hedge_rows)
hedge_freq_df.to_csv('hedge_frequency_stats.csv', index=False)

# Mis-spec test
mis_rows = []
for sym, info in market.items():
    S0 = info['S0']; sigma = info['sigma']; K = round(S0,2)
    T = 60/365
    for mult in [0.9, 1.1]:
        pnl, stats = hedge_pnl(S0, K, T, r, q, sigma_true=sigma, sigma_hedge=sigma*mult, n_paths=4000, steps=60, seed=3000+int(mult*100))
        mis_rows.append({'Symbol': sym, 'Sigma_true': sigma, 'Sigma_hedge': sigma*mult, **stats})
        plt.figure(figsize=(7.5,4))
        plt.hist(pnl, bins=60, alpha=0.8, color='teal', edgecolor='white')
        plt.title(f'Hedging P&L (σ mis-spec {mult:.1f}×) {sym}')
        plt.xlabel('P&L (premium included)')
        plt.ylabel('Frequency')
        plt.tight_layout()
        plt.savefig(f'hedge_misspec_hist_{sym}_{'m09' if mult<1 else 'p11'}.png', dpi=150)
        plt.close()

mis_df = pd.DataFrame(mis_rows)
mis_df.to_csv('hedge_misspec_stats.csv', index=False)

# Convergence vs BS at N=800 for a few maturities
tri_vs_bs_rows = []
for sym, info in market.items():
    S0 = info['S0']; sigma = info['sigma']; K = round(S0,2)
    for days in [30,60,90]:
        T = days/365
        tri = trinomial_log_price(S0, K, T, r, q, sigma, N=800, option='call', american=False)
        bs = bs_price(S0, K, T, r, q, sigma, 'call')
        tri_vs_bs_rows.append({'Symbol': sym, 'Days': days, 'Tri_LogSpace_N800': tri, 'BS': bs, 'Abs_Error': abs(tri-bs)})
tri_vs_bs_df = pd.DataFrame(tri_vs_bs_rows)
tri_vs_bs_df.to_csv('tri_vs_bs_convergence.csv', index=False)

# Return previews
n_sweep_preview = n_sweep_df.to_dict(orient='records')
tri_vs_bs_preview = tri_vs_bs_df.to_dict(orient='records')
hedge_freq_preview = hedge_freq_df.to_dict(orient='records')
mis_spec_preview = mis_df.to_dict(orient='records')

n_sweep_preview[:6], tri_vs_bs_preview, hedge_freq_preview, mis_spec_preview

([{'N': 25,
   'Trinomial_LogSpace_Price': 0.4006662120864776,
   'BS_Ref': 0.40440140615699605,
   'Abs_Error': 0.0037351940705184217},
  {'N': 50,
   'Trinomial_LogSpace_Price': 0.402543770302668,
   'BS_Ref': 0.40440140615699605,
   'Abs_Error': 0.0018576358543280214},
  {'N': 100,
   'Trinomial_LogSpace_Price': 0.40347497723790327,
   'BS_Ref': 0.40440140615699605,
   'Abs_Error': 0.000926428919092781},
  {'N': 200,
   'Trinomial_LogSpace_Price': 0.4039387772972486,
   'BS_Ref': 0.40440140615699605,
   'Abs_Error': 0.0004626288597474604},
  {'N': 400,
   'Trinomial_LogSpace_Price': 0.404170236726154,
   'BS_Ref': 0.40440140615699605,
   'Abs_Error': 0.00023116943084205976},
  {'N': 800,
   'Trinomial_LogSpace_Price': 0.404285857519487,
   'BS_Ref': 0.40440140615699605,
   'Abs_Error': 0.00011554863750906641}],
 [{'Symbol': 'SEA',
   'Days': 30,
   'Tri_LogSpace_N800': 0.404285857519487,
   'BS': 0.40440140615699605,
   'Abs_Error': 0.00011554863750906641},
  {'Symbol': 'SEA',
   'D

In [25]:
r=0.05; q=0.0; TRADING_DAYS=252
files={'SEA':'SEA.csv','AMKBY':'AMKBY.csv'}

def prep_df(df):
    df=df.copy(); df['Date']=pd.to_datetime(df['Date']); df=df.sort_values('Date')
    df['AdjClose']=pd.to_numeric(df.get('Adj Close', df['Close']), errors='coerce')
    for c in ['Open','High','Low','Close']: df[c]=pd.to_numeric(df[c], errors='coerce')
    df['overnight']=np.log(df['Open']/df['AdjClose'].shift(1))
    df['oc']=np.log(df['Close']/df['Open'])
    df['rs']=np.log(df['High']/df['Open'])*np.log(df['High']/df['Close']) + np.log(df['Low']/df['Open'])*np.log(df['Low']/df['Close'])
    return df

def yz(df):
    x=df.dropna(subset=['overnight','oc','rs']); n=len(x)
    if n<2: return np.nan
    k=0.34/(1.34+(n+1)/(n-1))
    var=x['overnight'].var(ddof=1)+k*x['oc'].var(ddof=1)+(1-k)*x['rs'].mean()
    return np.sqrt(TRADING_DAYS*var)

market={}
for s,f in files.items():
    df=pd.read_csv(f); df=prep_df(df)
    vols=[]
    for i in range(len(df)):
        lo=max(0,i-21+1); vols.append(yz(df.iloc[lo:i+1]))
    df['YZ_21d']=vols
    market[s]={'S0':float(df['AdjClose'].iloc[-1]), 'sigma':float(pd.to_numeric(df['YZ_21d']).dropna().iloc[-1])}

from math import log, sqrt

def bs_price(S,K,T,r,q,sigma,option='call'):
    if T<=0 or sigma<=0: return max(0.0, (S-K) if option=='call' else (K-S))
    d1=(np.log(S/K)+(r-q+0.5*sigma**2)*T)/(sigma*np.sqrt(T)); d2=d1-sigma*np.sqrt(T)
    if option=='call': return float(S*np.exp(-q*T)*norm.cdf(d1)-K*np.exp(-r*T)*norm.cdf(d2))
    else: return float(K*np.exp(-r*T)*norm.cdf(-d2)-S*np.exp(-q*T)*norm.cdf(-d1))


def logspace_params(dt,r,q,sigma):
    mu=(r-q-0.5*sigma**2); A=mu*dt; dx=sigma*np.sqrt(3*dt); B=sigma**2*dt + A*A
    pu=0.5*(A/dx + B/(dx*dx)); pd=0.5*(-A/dx + B/(dx*dx)); pm=1.0 - (B/(dx*dx))
    probs=np.array([pu,pm,pd]); probs=np.clip(probs,0,1); s=probs.sum(); probs=probs/s if s>0 else np.array([1/4,1/2,1/4])
    return dx, probs[0], probs[1], probs[2]


def tri_log_price(S0,K,T,r,q,sigma,N=200,option='call',american=False):
    dt=T/N; dx,pu,pm,pd=logspace_params(dt,r,q,sigma); disc=np.exp(-r*dt)
    ks=np.arange(-N,N+1); ST=S0*np.exp(ks*dx)
    V=np.maximum(0.0,ST-K) if option=='call' else np.maximum(0.0,K-ST)
    for i in range(N-1,-1,-1):
        ks=np.arange(-i,i+1)
        V_up=V[(ks+1)+(i+1)]; V_mid=V[(ks)+(i+1)]; V_down=V[(ks-1)+(i+1)]
        V=disc*(pu*V_up + pm*V_mid + pd*V_down)
        if american:
            S_i=S0*np.exp(ks*dx)
            exercise=np.maximum(0.0,S_i-K) if option=='call' else np.maximum(0.0,K-S_i)
            V=np.maximum(V,exercise)
    return float(V[0])

# N sweep
S0=market['SEA']['S0']; sigma=market['SEA']['sigma']; T=30/365; K=round(S0,2)
bs=bs_price(S0,K,T,r,q,sigma,'call')
Ns=[25,50,100,200,400,800]
ps=[tri_log_price(S0,K,T,r,q,sigma,N=n,option='call',american=False) for n in Ns]
result={'market':market,'bs':bs,'Ns':Ns,'tri_prices':ps}
result

{'market': {'SEA': {'S0': 16.0, 'sigma': 0.20299922907151258},
  'AMKBY': {'S0': 12.09, 'sigma': 0.46800149546516967}},
 'bs': 0.40440140615699605,
 'Ns': [25, 50, 100, 200, 400, 800],
 'tri_prices': [0.4006662120864776,
  0.402543770302668,
  0.40347497723790327,
  0.4039387772972486,
  0.404170236726154,
  0.404285857519487]}

In [26]:
try:
    r=0.05; q=0.0; TRADING_DAYS=252
    files={'SEA':'SEA.csv','AMKBY':'AMKBY.csv'}
    def prep_df(df):
        df=df.copy(); df['Date']=pd.to_datetime(df['Date']); df=df.sort_values('Date')
        df['AdjClose']=pd.to_numeric(df.get('Adj Close', df['Close']), errors='coerce')
        for c in ['Open','High','Low','Close']: df[c]=pd.to_numeric(df[c], errors='coerce')
        df['overnight']=np.log(df['Open']/df['AdjClose'].shift(1))
        df['oc']=np.log(df['Close']/df['Open'])
        df['rs']=np.log(df['High']/df['Open'])*np.log(df['High']/df['Close']) + np.log(df['Low']/df['Open'])*np.log(df['Low']/df['Close'])
        return df
    def yz(df):
        x=df.dropna(subset=['overnight','oc','rs']); n=len(x)
        if n<2: return np.nan
        k=0.34/(1.34+(n+1)/(n-1))
        var=x['overnight'].var(ddof=1)+k*x['oc'].var(ddof=1)+(1-k)*x['rs'].mean()
        return np.sqrt(TRADING_DAYS*var)
    market={}
    for s,f in files.items():
        df=pd.read_csv(f); df=prep_df(df)
        vols=[]
        for i in range(len(df)):
            lo=max(0,i-21+1); vols.append(yz(df.iloc[lo:i+1]))
        df['YZ_21d']=vols
        market[s]={'S0':float(df['AdjClose'].iloc[-1]), 'sigma':float(pd.to_numeric(df['YZ_21d']).dropna().iloc[-1])}
    print('MARKET',market)
except Exception as e:
    print('ERR',e)
    traceback.print_exc(file=sys.stdout)


MARKET {'SEA': {'S0': 16.0, 'sigma': 0.20299922907151258}, 'AMKBY': {'S0': 12.09, 'sigma': 0.46800149546516967}}


In [27]:
r=0.05; q=0.0; TRADING_DAYS=252
files={'SEA':'SEA.csv','AMKBY':'AMKBY.csv'}

def prep_df(df):
    df=df.copy(); df['Date']=pd.to_datetime(df['Date']); df=df.sort_values('Date')
    df['AdjClose']=pd.to_numeric(df.get('Adj Close', df['Close']), errors='coerce')
    for c in ['Open','High','Low','Close']: df[c]=pd.to_numeric(df[c], errors='coerce')
    df['overnight']=np.log(df['Open']/df['AdjClose'].shift(1))
    df['oc']=np.log(df['Close']/df['Open'])
    df['rs']=np.log(df['High']/df['Open'])*np.log(df['High']/df['Close']) + np.log(df['Low']/df['Open'])*np.log(df['Low']/df['Close'])
    return df

def yz(df):
    x=df.dropna(subset=['overnight','oc','rs']); n=len(x)
    if n<2: return np.nan
    k=0.34/(1.34+(n+1)/(n-1))
    var=x['overnight'].var(ddof=1)+k*x['oc'].var(ddof=1)+(1-k)*x['rs'].mean()
    return np.sqrt(TRADING_DAYS*var)

market={}
for s,f in files.items():
    df=pd.read_csv(f); df=prep_df(df)
    vols=[]
    for i in range(len(df)):
        lo=max(0,i-21+1); vols.append(yz(df.iloc[lo:i+1]))
    df['YZ_21d']=vols
    market[s]={'S0':float(df['AdjClose'].iloc[-1]), 'sigma':float(pd.to_numeric(df['YZ_21d']).dropna().iloc[-1])}

from math import log, sqrt

def bs_price(S,K,T,r,q,sigma,option='call'):
    if T<=0 or sigma<=0: return max(0.0, (S-K) if option=='call' else (K-S))
    d1=(np.log(S/K)+(r-q+0.5*sigma**2)*T)/(sigma*np.sqrt(T)); d2=d1-sigma*np.sqrt(T)
    if option=='call': return float(S*np.exp(-q*T)*norm.cdf(d1)-K*np.exp(-r*T)*norm.cdf(d2))
    else: return float(K*np.exp(-r*T)*norm.cdf(-d2)-S*np.exp(-q*T)*norm.cdf(-d1))


def logspace_params(dt,r,q,sigma):
    mu=(r-q-0.5*sigma**2); A=mu*dt; dx=sigma*np.sqrt(3*dt); B=sigma**2*dt + A*A
    pu=0.5*(A/dx + B/(dx*dx)); pd=0.5*(-A/dx + B/(dx*dx)); pm=1.0 - (B/(dx*dx))
    probs=np.array([pu,pm,pd]); probs=np.clip(probs,0,1); s=probs.sum(); probs=probs/s if s>0 else np.array([1/4,1/2,1/4])
    return dx, probs[0], probs[1], probs[2]


def tri_log_price(S0,K,T,r,q,sigma,N=200,option='call',american=False):
    dt=T/N; dx,pu,pm,pd=logspace_params(dt,r,q,sigma); disc=np.exp(-r*dt)
    ks=np.arange(-N,N+1); ST=S0*np.exp(ks*dx)
    V=np.maximum(0.0,ST-K) if option=='call' else np.maximum(0.0,K-ST)
    for i in range(N-1,-1,-1):
        ks=np.arange(-i,i+1)
        V_up=V[(ks+1)+(i+1)]; V_mid=V[(ks)+(i+1)]; V_down=V[(ks-1)+(i+1)]
        V=disc*(pu*V_up + pm*V_mid + pd*V_down)
        if american:
            S_i=S0*np.exp(ks*dx)
            exercise=np.maximum(0.0,S_i-K) if option=='call' else np.maximum(0.0,K-S_i)
            V=np.maximum(V,exercise)
    return float(V[0])

S0=market['SEA']['S0']; sigma=market['SEA']['sigma']; T=30/365; K=round(S0,2)
bs=bs_price(S0,K,T,r,q,sigma,'call')
Ns=[25,50,100,200]
ps=[tri_log_price(S0,K,T,r,q,sigma,N=n,option='call',american=False) for n in Ns]
{'market':market,'bs':bs,'Ns':Ns,'tri_prices':ps}


{'market': {'SEA': {'S0': 16.0, 'sigma': 0.20299922907151258},
  'AMKBY': {'S0': 12.09, 'sigma': 0.46800149546516967}},
 'bs': 0.40440140615699605,
 'Ns': [25, 50, 100, 200],
 'tri_prices': [0.4006662120864776,
  0.402543770302668,
  0.40347497723790327,
  0.4039387772972486]}